In [1]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
# 이미지 로드
def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img , (256,256))
    img = img / 255.0
    return img

In [3]:
# json 파일 로드
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    age = data['info']['age']
    gender = 1 if data['info']['gender'] =='F' else 0 
    skin_type = data['info']['skin_type']
    sensitive = data['info']['sensitive']

    pigmentation = data['annotations']['forehead_pigmentation']
    
    return age, gender, skin_type, sensitive, pigmentation

In [7]:
from sklearn.model_selection import train_test_split
import os
import os

image_dir = r"D:\data\korean 01 data"
json_dir = r"D:\data\jsonfile"

images = []
metadata = []

four_or_five_path = []

for main_folder in os.listdir(image_dir):  # 'digit cam', 'pad', 'cell phone'
    main_folder_path = os.path.join(image_dir, main_folder)
    if os.path.isdir(main_folder_path):  # 메인 폴더인지 확인
        for sub_folder in os.listdir(main_folder_path):  # '0001', '0002', '0003'
            sub_folder_path = os.path.join(main_folder_path, sub_folder)
            if os.path.isdir(sub_folder_path):  # 서브 폴더인지 확인
                for filename in os.listdir(sub_folder_path):  # 이미지 파일들
                    if filename.endswith('.jpg'):  # .jpg 파일만 처리
                        image_path = os.path.join(sub_folder_path, filename)
                        
                        # 여기서 특정 조건에 맞는 파일만 선택하도록 수정
                        if 'F' not in filename:  # L15가 포함된 파일만 선택
                            continue  # L15가 포함되지 않으면 건너뜀


                        # JSON 파일명 생성: 'cropped_' 제거하고 '.jpg' -> '.json'
                        json_filename = filename.replace('cropped_', '').replace('.jpg', '')
                        
                        # json_dir도 image_dir처럼 동일한 폴더 구조를 반영하여 경로 설정
                        json_folder_path = os.path.join(json_dir, main_folder, sub_folder)
                        json_path = os.path.join(json_folder_path, json_filename)  # JSON 파일 경로
                        

                        # 이미지와 JSON 로드
                        image = load_image(image_path)
                        if image is None:
                            continue  # 이미지 로드 실패시 건너뛰기
                        age, gender, skin_type, sensitive, pigmentation = load_json(json_path)

                        if pigmentation == 4 or pigmentation == 5:
                            four_or_five_path.append(json_path)
                            pigmentation = 3
                        
                        images.append(image)  # images 리스트에 추가
                        metadata.append([age, gender, skin_type, sensitive, pigmentation])  # metadata 리스트에 추가



# 리스트를 numpy 배열로 변환
images = np.array(images)
metadata = np.array(metadata)

X = images
y = metadata[:,-1] # target

X_metadata = metadata[:, :-1] # pigment 제외한 나머지 데이터


X_train,X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 42)

X_train_images = X_train
X_val_images = X_val

X_train_metadata = X_metadata[:len(X_train)]
X_val_metadata = X_metadata[len(X_train):]

In [13]:
import random

index_list = random.sample(range(1, 100), 50)
df.iloc[index_list, :]

[76,
 26,
 6,
 30,
 77,
 20,
 82,
 73,
 87,
 15,
 44,
 8,
 16,
 91,
 67,
 43,
 62,
 56,
 84,
 18,
 9,
 36,
 59,
 55,
 21,
 90,
 48,
 81,
 96,
 88,
 54,
 12,
 23,
 13,
 68,
 24,
 93,
 51,
 66,
 17,
 72,
 33,
 95,
 63,
 25,
 85,
 47,
 14,
 92,
 37]

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input


image_input = Input(shape=(256,256,3), name= 'image_input')
x = layers.Conv2D(256,(3,3), activation = 'relu')(image_input)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(128,(3,3), activation = 'relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64,(3,3), activation = 'relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(32,(3,3), activation = 'relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)


metadata_input = Input(shape=(4,), name='metadata_input')
y = layers.Dense(32, activation ='relu')(metadata_input)
y = layers.Dense(16, activation='relu')(y)

combined = layers.concatenate([x,y])
output = layers.Dense(4, activation='softmax')(combined)

model = models.Model(inputs =[image_input, metadata_input], outputs=output)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_5 (Conv2D)           (None, 254, 254, 256)        7168      ['image_input[0][0]']         
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 127, 127, 256)        0         ['conv2d_5[0][0]']            
 g2D)                                                                                             
                                                                                                  
 conv2d_6 (Conv2D)           (None, 125, 125, 128)        295040    ['max_pooling2d_4[0][0]'

In [ ]:
# 사전학습 모델 
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

image_input = Input(shape=(224, 224, 3), name='image_input')

# ResNet50을 사전학습된 가중치로 불러옴 (include_top=False는 분류용 마지막 레이어 제외)
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))(image_input)
x = layers.GlobalAveragePooling2D()(resnet)  # ResNet50의 출력은 GlobalAveragePooling2D로 처리

# 메타데이터 입력
metadata_input = Input(shape=(4,), name='metadata_input')
y = layers.Dense(32, activation='relu')(metadata_input)
y = layers.Dense(16, activation='relu')(y)

# 두 입력을 결합
combined = layers.concatenate([x, y])

# 최종 출력 레이어
output = layers.Dense(4, activation='softmax')(combined)

# 모델 정의
model2 = Model(inputs=[image_input, metadata_input], outputs=output)

# 모델 컴파일
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 요약
model2.summary()


In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# 조기 종료 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',  # val_loss를 모니터링
    patience=3,          # 개선되지 않은 에포크를 3번 참음
    restore_best_weights=True  # 최상의 모델 가중치를 복원
)

In [17]:
# 모델 학습
history = model.fit(
    {'image_input': X_train_images, 'metadata_input': X_train_metadata},  # 학습 데이터
    y_train,  # 실제 레이블 (pigmentation 값)
    epochs=20,  # 에포크 수
    validation_data=(
        {'image_input': X_val_images, 'metadata_input': X_val_metadata},  # 검증 데이터
        y_val  # 검증 데이터의 실제 레이블 (pigmentation 값)
    ),
    callbacks = [early_stopping]
)

# 학습이 완료되면 학습 과정에서의 손실과 메트릭스를 확인할 수 있습니다.
print("Training History: ", history.history)


Epoch 1/20


73/73 [==============================] - 679s 9s/step - loss: 1.3158 - accuracy: 0.4849 - val_loss: 1.2466 - val_accuracy: 0.5233
Epoch 2/20
73/73 [==============================] - 647s 9s/step - loss: 1.2355 - accuracy: 0.5086 - val_loss: 1.2188 - val_accuracy: 0.5233
Epoch 3/20
73/73 [==============================] - 646s 9s/step - loss: 1.2262 - accuracy: 0.5082 - val_loss: 1.2030 - val_accuracy: 0.5233
Epoch 4/20
73/73 [==============================] - 647s 9s/step - loss: 1.2239 - accuracy: 0.5091 - val_loss: 1.1994 - val_accuracy: 0.5233
Epoch 5/20
73/73 [==============================] - 651s 9s/step - loss: 1.2197 - accuracy: 0.5091 - val_loss: 1.2051 - val_accuracy: 0.5233
Epoch 6/20
73/73 [==============================] - 649s 9s/step - loss: 1.2227 - accuracy: 0.5091 - val_loss: 1.2141 - val_accuracy: 0.5233
Epoch 7/20
73/73 [==============================] - 651s 9s/step - loss: 1.2225 - accuracy: 0.5091 - val_loss: 1.2325 - val_accuracy: 0.5233
Training Hi

In [18]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(
    {'image_input': X_val_images, 'metadata_input': X_val_metadata},  # 검증 데이터
    y_val  # 검증 데이터의 실제 레이블
)

print("Test Loss: ", test_loss)
print("Test Accuracy: ", test_accuracy)

# 0.58 0.56
# 3으로 합친 후 : 0.569, stratify = y : 0.538
# 모든 데이터 합친 후 : 0.52

19/19 [==============================] - 27s 1s/step - loss: 1.1994 - accuracy: 0.5233
Test Loss:  1.1994446516036987
Test Accuracy:  0.5233160853385925


In [10]:
import pandas as pd
columns = ['Age', 'Gender', 'Skin_type', 'Sensitive','Pigmentation']
metadata_df = pd.DataFrame(metadata, columns =columns)

metadata_df

Age  Gender  Skin_type  Sensitive  Pigmentation
0      55       1          3          0             1
1      50       1          0          0             1
2      24       1          0          0             1
3      47       1          4          1             1
4      55       1          3          0             1
...   ...     ...        ...        ...           ...
2890   25       1          1          0             1
2891   24       1          1          0             1
2892   23       1          1          0             1
2893   26       1          1          1             1
2894   28       1          3          1             1

[2895 rows x 5 columns]

In [11]:
metadata_df['Pigmentation'].value_counts()

Pigmentation
1    1482
0     588
2     552
3     273
Name: count, dtype: int64